# Load packages and data

In [ ]:
packages <- c("tidyverse", "phyloseq", "microbiome", "viridis", "corrplot", "ggcorrplot")
invisible(lapply(packages, function(p) suppressPackageStartupMessages(library(p, character.only = TRUE))))

In [ ]:
options(repr.plot.width=20, repr.plot.height=25)

In [ ]:
# import data
nasal_phylo_clr <- readRDS("../../results/Microbiome_analysis/nasal_samples_clean_clr.rds")
nasal_phylo_clr

In [ ]:
# import data
gut_phylo_clr <- readRDS("../../results/Microbiome_analysis/gut_samples_clean_clr.rds")
gut_phylo_clr

In [ ]:
genera_of_interest_nasal <- c("Staphylococcus", "Dolosigranulum", "Corynebacterium", "Lactobacillus", "Lawsonella", "Cutibacterium")
genera_of_interest_gut <- c("Bifidobacterium", "Collinsella", "Bacteroides", "Ruminococcus", "Prevotella", "Faecalibacterium")

# Functions

In [ ]:
create_subset_data <- function(phylo_obj, input_timepoint, input_treatment, genera_of_interest) {
	# Create subset based on timepoint and treatment
	genera_subset <- prune_samples(sample_data(phylo_obj)$timepoint == input_timepoint, phylo_obj)
	genera_subset <- prune_samples(sample_data(genera_subset)$treatment %in% input_treatment, genera_subset)
	
	# Get taxa table and find matching genera
	taxa_table <- tax_table(genera_subset)
	matching_taxa <- rownames(taxa_table)[taxa_table[, "Genus"] %in% genera_of_interest]
	
	# Subset taxa and aggregate rare taxa
	genera_subset <- prune_taxa(matching_taxa, genera_subset)
	genera_subset <- aggregate_rare(genera_subset, level = "Genus", detection = 1/100, prevalence = 1/100)
	
	# Convert OTU table to data frame
	otu_genera_subset <- genera_subset %>%
			otu_table() %>%
			t() %>%
			data.frame() %>%
			rownames_to_column("Sample_ID")
	
	# Get sample data
	sample_data <- sample_data(phylo_obj) %>%
			data.frame() %>%
			rownames_to_column("Sample_ID") %>%
			select(-sequencing_run, -treatment, -patient, -timepoint, -Sex_male, -Smoking)
	
	# Combine OTU and sample data
	combined_data <- inner_join(otu_genera_subset, sample_data, by = "Sample_ID") %>%
			column_to_rownames("Sample_ID")
			
	return(combined_data)  
}

In [ ]:
plot_corr_plotting <- function(corr_matrix, p_matrix, genera_of_interest, title, save_pdf = FALSE){
	# Select only the genera of interest and clinical factors
	corr_matrix <- corr_matrix[genera_of_interest, !colnames(corr_matrix) %in% genera_of_interest]
	p_matrix <- p_matrix[genera_of_interest, !colnames(p_matrix) %in% genera_of_interest]

	# If saving to PDF, start PDF device
	if(save_pdf) {
			# Create directory if it doesn't exist
			save_path <- "../../results/Microbiome_analysis/plots/correlations"
			if (!dir.exists(save_path)) {
					dir.create(save_path, recursive = TRUE)
			}
			
			# Create filename from title
			filename <- file.path(save_path, 
													paste0(gsub(" ", "_", tolower(title)), ".pdf"))
			pdf(filename, width = 20, height = 10)
			
			# Create the plot for PDF
			par(mar = c(0,0,2,0))    
			main_title <- title.cex <- 2  
			plot <- corrplot(corr_matrix,
					method = "color", 
					type = "full",
					order = "original",
					tl.col = "black",
					tl.srt = 90,
					p.mat = p_matrix,
					number.cex = 1.2,     
					tl.cex = 1.2,
					cl.cex = 1.2,         
					col = viridis(200, option = 'H'),  
					addCoef.col = "white",
					insig = "blank",
					addgrid.col = "black",  
					outline = TRUE,
					mar = c(0,0,4,0)    
			)
			title(main = title, cex.main = main_title)
			dev.off()
			cat("Saved:", filename, "\n")
	}

	# Create the plot for display
	options(repr.plot.width=20, repr.plot.height=10)
	par(mar = c(0,0,2,0))    
	main_title <- title.cex <- 2  
	plot <- corrplot(corr_matrix,
			method = "color", 
			type = "full",
			order = "original",
			tl.col = "black",
			tl.srt = 90,
			p.mat = p_matrix,
			number.cex = 1.2,     
			tl.cex = 1.2,
			cl.cex = 1.2,         
			col = viridis(200, option = 'H'),  
			addCoef.col = "white",
			insig = "blank",
			addgrid.col = "black",  
			outline = TRUE,
			mar = c(0,0,4,0)    
	)
	
	# Add title separately using base R
	title(main = title, cex.main = main_title)

	return(plot)
}

# Nasal pasage Dupilumab treatment

### day 0

In [ ]:
nasal_dupilumab_day_0 <- create_subset_data(
	phylo_obj = nasal_phylo_clr,
	input_timepoint = 0,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_nasal
)

In [ ]:
nasal_dupilumab_rho_0 <- cor(nasal_dupilumab_day_0, method = "spearman", use = "pairwise.complete.obs")
nasal_dupilumab_p_0 <- cor_pmat(nasal_dupilumab_day_0, method = "spearman",  exact = FALSE)
nasal_dupilumab_p_adjusted_0 <- apply(nasal_dupilumab_p_0, 2, p.adjust, method = "fdr")

In [ ]:
nasal_dupilumab_0_plot <- plot_corr_plotting(
	nasal_dupilumab_rho_0, 
	nasal_dupilumab_p_adjusted_0, 
	genera_of_interest_nasal, 
	"Spearman correlation cytokines and selected nasal passage genera (Dupilumab treatment day 0)",
	save_pdf = TRUE
)

### day 28

In [ ]:
nasal_dupilumab_day_28 <- create_subset_data(
	phylo_obj = nasal_phylo_clr,
	input_timepoint = 28,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_nasal
)

In [ ]:
nasal_dupilumab_rho_28 <- cor(nasal_dupilumab_day_28, method = "spearman", use = "pairwise.complete.obs")
nasal_dupilumab_p_28 <- cor_pmat(nasal_dupilumab_day_28, method = "spearman",  exact = FALSE)
nasal_dupilumab_p_adjusted_28 <- apply(nasal_dupilumab_p_28, 2, p.adjust, method = "fdr")

In [ ]:
nasal_dupilumab_28_plot <- plot_corr_plotting(
	nasal_dupilumab_rho_28, 
	nasal_dupilumab_p_adjusted_28, 
	genera_of_interest_nasal, 
	"Spearman correlation cytokines and selected nasal passage genera (Dupilumab treatment day 28)",
	save_pdf = TRUE
)

### day 90

In [ ]:
nasal_dupilumab_day_90 <- create_subset_data(
	phylo_obj = nasal_phylo_clr,
	input_timepoint = 90,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_nasal
)

In [ ]:
nasal_dupilumab_rho_90 <- cor(nasal_dupilumab_day_90, method = "spearman", use = "pairwise.complete.obs")
nasal_dupilumab_p_90 <- cor_pmat(nasal_dupilumab_day_90, method = "spearman",  exact = FALSE)
nasal_dupilumab_p_adjusted_90 <- apply(nasal_dupilumab_p_90, 2, p.adjust, method = "fdr")

In [ ]:
nasal_dupilumab_90_plot <- plot_corr_plotting(
	nasal_dupilumab_rho_90, 
	nasal_dupilumab_p_adjusted_90, 
	genera_of_interest_nasal, 
	"Spearman correlation cytokines and selected nasal passage genera (Dupilumab treatment day 90)",
	save_pdf = TRUE
)

# Gut samples Dupilumab treatment

### day 0

In [ ]:
gut_dupilumab_day_0 <- create_subset_data(
	phylo_obj = gut_phylo_clr,
	input_timepoint = 0,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_gut
)

In [ ]:
gut_dupilumab_rho_0 <- cor(gut_dupilumab_day_0, method = "spearman", use = "pairwise.complete.obs")
gut_dupilumab_p_0 <- cor_pmat(gut_dupilumab_day_0, method = "spearman",  exact = FALSE)
gut_dupilumab_p_adjusted_0 <- apply(gut_dupilumab_p_0, 2, p.adjust, method = "fdr")

In [ ]:
gut_dupilumab_0_plot <- plot_corr_plotting(
	gut_dupilumab_rho_0, 
	gut_dupilumab_p_adjusted_0, 
	genera_of_interest_gut, 
	"Spearman correlation cytokines and selected gut samples genera (Dupilumab treatment day 0)",
	save_pdf = TRUE
)

### day 28

In [ ]:
gut_dupilumab_day_28 <- create_subset_data(
	phylo_obj = gut_phylo_clr,
	input_timepoint = 28,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_gut
)

In [ ]:
gut_dupilumab_rho_28 <- cor(gut_dupilumab_day_28, method = "spearman", use = "pairwise.complete.obs")
gut_dupilumab_p_28 <- cor_pmat(gut_dupilumab_day_28, method = "spearman",  exact = FALSE)
gut_dupilumab_p_adjusted_28 <- apply(gut_dupilumab_p_28, 2, p.adjust, method = "fdr")

In [ ]:
gut_dupilumab_28_plot <- plot_corr_plotting(
	gut_dupilumab_rho_28, 
	gut_dupilumab_p_adjusted_28, 
	genera_of_interest_gut, 
	"Spearman correlation cytokines and selected gut samples genera (Dupilumab treatment day 28)",
	save_pdf = TRUE
)

### Day 90

In [ ]:
gut_dupilumab_day_90 <- create_subset_data(
	phylo_obj = gut_phylo_clr,
	input_timepoint = 90,
	input_treatment = "Dupilumab_treatment",
	genera_of_interest = genera_of_interest_gut
)

In [ ]:
gut_dupilumab_rho_90 <- cor(gut_dupilumab_day_90, method = "spearman", use = "pairwise.complete.obs")
gut_dupilumab_p_90 <- cor_pmat(gut_dupilumab_day_90, method = "spearman",  exact = FALSE)
gut_dupilumab_p_adjusted_90 <- apply(gut_dupilumab_p_90, 2, p.adjust, method = "fdr")

In [ ]:
gut_dupilumab_90_plot <- plot_corr_plotting(
	gut_dupilumab_rho_90, 
	gut_dupilumab_p_adjusted_90, 
	genera_of_interest_gut, 
	"Spearman correlation cytokines and selected gut samples genera (Dupilumab treatment day 90)",
	save_pdf = TRUE
)